In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
import xml.etree.ElementTree as et
from io import BytesIO
from typing import List, Dict, Optional

# Классы и методы

In [ ]:
class FedStatIndicator:
    def __init__(self, indicator_id):
        self.id = indicator_id
        self._filters_raw = None
        self._raw_data = None

    def _get_filters_raw(self):
        """Получает сырые данные фильтров"""
        html = requests.get(f'https://www.fedstat.ru/indicator/{self.id}')
        if html.status_code == 200:
            soup = BeautifulSoup(html.text, "lxml")
            script = soup.find_all("script")[11].text
            pattern = r"filters:\s*(\{.*?\})(?=\s*,\s*left_columns)"
            match = re.search(pattern, script, re.DOTALL).group(0)
            filters = "{" + match + "}"
            filters = re.sub(r'([{,]\s*)(\w+)(\s*:)', r'\1"\2"\3', filters)
            filters = filters.replace("'", '"')
            self._filters_raw = json.loads(filters)['filters']
            return self._filters_raw
        else:
            raise requests.RequestException(f"Ошибка HTTP-запроса")

    def get_filters_text(self):
        """
        Возвращает код и название показателя, доступного для фильтрации
        """
        filters = self._get_filters_raw()
        filter_values = {key : filters[key]['title'] for key in list(filters.keys())[1:]}
        return filter_values

    def get_filter_values(self):
        """
        Возвращает названия и коды доступных значений для выбранных фильтров
        """
        filters = self._get_filters_raw()
        filter_codes = self.get_filters_text()
        
        categories =[]
        for key in filter_codes.keys():
            categories.append({
                key: list(filters[key]['values'].keys())
            })
        filter_ids = [f"{k}_{val}" for item in categories for k, v in item.items() for val in v]
        return filter_ids

    def load_raw_indicator(self, data_type: str = "excel", filter_ids: List["str"] =  None):
        """
       Загружает данные индикатора через API

       :param data_type: тип возвращаемых данных
       :param filter_ids: список с кодами выбранных показателей
       :return: DataFrame с данными
        """
        if filter_ids is None:
            filter_ids = self.get_filter_values()
        data = {
                "lineObjectIds": ["0",  "57831", "58335", "58274", "30611"],
                "columnObjectIds": ["3", "33560"],
                "selectedFilterIds": [
                filter_ids
                ]
            }
        params = {
            "format" : data_type,
            "id" : self.id  
        }
        try: 
            response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
            if "excel" in response.headers.get("Content-Type"):
                self._raw_data = pd.read_excel(BytesIO(response.content), engine = "xlrd") 
                return self._raw_data
            else:
                raise ValueError(
                    f"Ответ не содержит {data_type} файла"
                )
        except requests.RequestException as e:
            raise requests.RequestException(f"Ошибка HTTP-запроса: {str(e)}")

    @staticmethod
    def _get_min_age(age_str):
        numbers = re.findall(r'\d+', age_str)
        return int(numbers[0]) if numbers else None

    @staticmethod
    def _get_max_age(age_str):
        numbers = re.findall(r"\d+", age_str)
        return int(numbers[-1]) if numbers else None

    @staticmethod   
    def _categorize_age(age_str):
        age_str = age_str.lower()
        numbers = re.findall(r'\d+', age_str)
        words = re.findall(r'\b[а-яА-ЯёЁ]+\b', age_str, flags=re.IGNORECASE)
        if "-" in age_str and len(words) == 1:
            age_diff = int(numbers[1]) - int(numbers[0])
            return 2 if age_diff == 4 else 3
        elif "всего" in age_str or not numbers or len(words) > 3:
            return 4
        else:
            return 1
    
    def _preprocess_dataframe(self, df):
     
        df.drop([df.columns[0], df.columns[4]], axis = 1, inplace = True)
        df = df.drop(0, axis = 0).reset_index(drop = True)

        df.rename(columns = {
            'Unnamed: 1' : "region",
            "Unnamed: 2" : "age",
            "Unnamed: 3" : "settlement"
            }, inplace = True)

        df.region = df.region.str.strip()
        years = [col for col in df.columns if col.isdigit()]
        df[years] = df[years].astype("Int64")
        
        return df

    def _change_districts(self, df):
        
        rm_districts = '|'.join([
            r'Южный федеральный',
            r'Сибирский федеральный',
            r'Северо-Кавказский федеральный',
            r'Дальневосточный федеральный'
        ])

        districts_mask = df['region'].str.contains(rm_districts, case=False)
        df= df[~districts_mask].reset_index(drop = True)

        dfo_regions = [
            'Республика Бурятия',
            'Забайкальский край',
            'Республика Саха (Якутия)',
            'Камчатский край',
            'Приморский край',
            'Хабаровский край',
            'Амурская область',
            'Магаданская область',
            'Сахалинская область',
            'Еврейская автономная область',
            'Чукотский автономный округ'
        ]
        dfo_mask = df['region'].isin(dfo_regions)
        dfo = df[dfo_mask].copy()
        dfo_agg = dfo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
        dfo_agg.insert(0, 'region', 'Дальневосточный федеральный округ')

        sfo_regions = ['Республика Алтай',
            'Республика Тыва',
            'Республика Хакасия',
            'Алтайский край',
            'Красноярский край',
            'Иркутская область',
            'Кемеровская область - Кузбасс',
            'Новосибирская область',
            'Омская область',
            'Томская область']
        sfo_mask = df['region'].isin(sfo_regions)
        sfo = df[sfo_mask].copy()
        sfo_agg = sfo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
        sfo_agg.insert(0, 'region', 'Сибирский федеральный округ')
        
        ufo_pattern = '|'.join([
            r'Адыгея', 
            r'Калмыкия',
            r'Краснодарский',
            r'Астраханская',
            r'Волгоградская',
            r'Ростовская',
            r'Крым',
            r'Севастополь'
        ])

        skfo_pattern = '|'.join([
            r'Дагестан',
            r'Ингушетия',
            r'Кабардин',
            r'Балкар',
            r'Карачаев',
            r'Черкес',
            r'Осетия',
            r'Алания',
            r'Чечен',
            r'Ставрополь'
        ])

        ufo_mask = df['region'].str.contains(ufo_pattern, case = False, na = False)
        ufo = df[ufo_mask].copy()
        ufo_agg = ufo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
        ufo_agg.insert(0, 'region', 'Южный федеральный округ')

        skfo_mask = df['region'].str.contains(skfo_pattern, case = False, na = False)
        skfo = df[skfo_mask].copy()
        skfo_agg = skfo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
        skfo_agg.insert(0, 'region', 'Северо-Кавказский федеральный округ')

        df_mid = pd.concat([df, ufo_agg, skfo_agg, sfo_agg], ignore_index = True)
        
        return df_mid

    def _add_mid_year_values(self, df):
        
        df['min_age']  = df['age'].apply(self._get_min_age)
        df['max_age'] = df['age'].apply(self._get_max_age)
        df['age_category'] = df['age'].apply(self._categorize_age)

        df = df.groupby('region', sort = False).apply(
            lambda x: x.sort_values(['age_category', 'min_age', 'max_age'])
        ).reset_index(drop = True)
        
        df.drop(columns = ['age_category', 'min_age', 'max_age'], inplace = True)
        return df

    def get_processed_data(self, data_type: str = "excel", filter_ids: List["str"] =  None ):

        raw_df = self.load_raw_indicator(
            data_type = data_type,
            filter_ids = filter_ids
        )
        df = self._preprocess_dataframe(raw_df)
        # df = self._change_districts(df)
        # df = self._add_mid_year_values(df)
        return df

In [ ]:
#Load the data
def get_filters_raw(id):
    """
    Код принимает на вход id показателя и возвращает список с фильтрами
    Это вспомогательная функция
    """
    try: 
        html = requests.get(f'https://www.fedstat.ru/indicator/{id}')
        soup = BeautifulSoup(html.text, "lxml")
        script = soup.find_all("script")[11].text
        pattern = r"filters:\s*(\{.*?\})(?=\s*,\s*left_columns)"
        match = re.search(pattern, script, re.DOTALL).group(0)
        filters = "{" + match + "}"
        filters = re.sub(r'([{,]\s*)(\w+)(\s*:)', r'\1"\2"\3', filters)
        filters = filters.replace("'", '"')
        filters = json.loads(filters)['filters']
        return filters
    except requests.RequestException as e:
        raise requests.RequestException(f"Ошибка HTTP-запроса: {str(e)}")

def get_filters_text(filters):
    """
    Возвращает код и название показателя, доступного для фильтрации
    """
    filter_values = {key : filters[key]['title'] for key in list(filters.keys())[1:]}
    return filter_values

def get_values(filters):
    """
    Возвращает названия и коды доступных значений для выбранных фильтров
    """
    filter_codes = filters.keys()
    categories =[]
    for key in filter_codes:
        categories.append({
            key: list(filters[key]['values'].keys())
        })
    ids = [f"{k}_{val}" for item in categories for k, v in item.items() for val in v]
    return ids

def load_indicator(id, data_type, ids):
    """
    На вход передается код индикатора, тип данных, коды показателей для фильтрации
    Возвращает датафрейм с данными
    """
    data = {
            "lineObjectIds": ["0",  "57831", "58335", "58274", "30611"],
            "columnObjectIds": ["3", "33560"],
            "selectedFilterIds": [
                ids
            ]
        }
    params = {
        "format" : data_type,
        "id" : id
    }
    try: 
        response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
        if "excel" in response.headers.get("Content-Type"):
            return pd.read_excel(BytesIO(response.content), engine = "xlrd")
        else:
            raise ValueError(
                f"Ответ не содержит {data_type} файла"
            )
    except requests.RequestException as e:
        raise requests.RequestException(f"Ошибка HTTP-запроса: {str(e)}")  

# Женское население

## SDMX data

In [ ]:
#All values
data = {
        "lineObjectIds": ["0", "30611", "58335", "57831", "58274"],
        "columnObjectIds": ["3", "33560"],
        "selectedFilterIds": ids
}
params = {
    "format" : "sdmx",
    "id" : "33459"
}
response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
print(response.status_code, response.headers)

In [7]:
# Adding  regions
regions = [f"{k}_{val}" for item in categories for k, v in item.items() if k =="57831" for val in v]

In [8]:
#Adding years
years = [f"{k}_{val}" for item in categories for k, v in item.items() if k =="3" for val in v]

In [9]:
#Adding settlement type
settlement_type = [f"{k}_{val}" for item in categories for k, v in item.items() if k =="58274" for val in v]
settlement_type

['58274_1744150', '58274_1750789', '58274_1750788']

In [10]:
#Adding age categories
age =  [f"{k}_{val}" for item in categories for k, v in item.items() if k =="58335" for val in v]

In [12]:
#collect regional data
data = {
        "lineObjectIds": ["0", "30611", "58335", "57831", "58274"],
        "columnObjectIds": ["3", "33560"],
        "selectedFilterIds": [
            "0_33459",
            years,
            "30611_950458",
            "33560_1540248",
            regions,
            "58274_1744150",
            "58335_1709566"
        ]
    }
params = {
    "format" : "sdmx",
    "id" : "33459"
}
response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
print(response.status_code, response.headers)

200 {'Content-Type': 'text/xml', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Sat, 12 Jul 2025 10:39:48 GMT', 'Set-Cookie': 'JSESSIONID=20E85FCD89FE3297FE03E7FAF9608248; Path=/; Secure; HttpOnly, session-cookie=18517ad4dc4c6a7edcd5a505204218c4d2506ea5bd73d1c7df057e3ecf38e193bc79ec9cdd66ebf6efc4768835a5ee4b; Max-Age=86400; Path=/; secure; HttpOnly', 'Cache-Control': 'no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': '0', 'Pragma': 'public', 'Content-Disposition': "attachment; filename*=UTF-8''data.xml", 'vary': 'accept-encoding', 'Content-Language': 'ru', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'Content-Security-Policy': 'upgrade-insecure-requests', 'Referrer-Policy': 'strict-origin'}


In [13]:
# Save the data in a file
with open("regions.xml", "wb") as file:
    file.write(response.content)

In [41]:
#Transform sdmx file into pandas dataframe
def read_sdmx(filename):
    tree = et.parse(filename)
    root = tree.getroot()

    namespaces = {
        "generic": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/generic",
        "message": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/message",
        "structure": 'http://www.SDMX.org/resources/SDMXML/schemas/v1_0/structure'
    } 

    dataset = root.find(".//message:DataSet", namespaces)
    data = []
    for series in dataset.findall(".//generic:Series", namespaces):
        observations = series.findall(".//generic:Obs", namespaces)
        pair = {}
        for obs in observations:
            time = obs.find(".//generic:Time", namespaces).text
            value = obs.find(".//generic:ObsValue", namespaces).attrib['value']
            region = obs.find(".//")
            data.append({
                "year" : time,
                "population" : value
                })
    return pd.DataFrame(data)

data = read_sdmx("regions.xml")
data

,year,population
0,1990,78549601
1,1991,78817695
2,1992,78914788
3,1993,78899303
4,1994,78770458
...,...,...
3502,1996,3833075
3503,1999,3700204
3504,2000,3620686
3505,2001,3620686


## Excel data

In [40]:
#Let's get the data in excel
def load_indicator(id, data_type, regions, years, settlement_type, age):
    data = {
            "lineObjectIds": ["0",  "57831", "58335", "58274", "30611"],
            "columnObjectIds": ["3", "33560"],
            "selectedFilterIds": [
                regions,
                "0_33459",
                years,
                "30611_950458",
                "33560_1540248",
                settlement_type,
                age
            ]
        }
    params = {
        "format" : data_type,
        "id" : id
    }
    response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
    return response
# print(response.status_code, response.headers)

In [7]:
#Load the data
df = pd.read_excel("all_data.xls", header = 2)
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,1990,1991,1992,1993,1994,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,NaN,NaN,NaN,NaN,NaN,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,...,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января
1,Численность постоянного населения - женщин по ...,Российская Федерация,Всего,все население,человек,78549601,78817695,78914788,78899303,78770458,...,78958558,79137859,79207619,79202094,79219390,78952485,78601633,78354712,78233436,78154709
2,Численность постоянного населения - женщин по ...,Российская Федерация,Всего,городское население,человек,57983496,58303864,58267844,57878461,57688561,...,59279662,59512332,59660317,59755068,59858643,59691444,59487917,59352665,59312967,59399259
3,Численность постоянного населения - женщин по ...,Российская Федерация,Всего,сельское население,человек,20566105,20513831,20646944,21020842,21081897,...,19678896,19625527,19547302,19447026,19360747,19261041,19113716,19002047,18920469,18755450
4,Численность постоянного населения - женщин по ...,Российская Федерация,0,все население,человек,1035959,957048,868672,770771,675810,...,942238,910777,814676,771071,714593,688397,673494,632575,611254,593327


In [8]:
#Remove unnecassary columns
df.drop([df.columns[0], df.columns[4]], axis = 1, inplace = True)
df = df.drop(0, axis = 0).reset_index(drop = True)

In [9]:
# Rename
df.rename(columns = {
    'Unnamed: 1' : "region",
    "Unnamed: 2" : "age",
    "Unnamed: 3" : "settlement"
    }, inplace = True)

In [10]:
df.region = df.region.str.strip()

In [11]:
#Remove unnecessary districts
rm_districts = '|'.join([
    r'Южный федеральный',
    r'Сибирский федеральный',
    r'Северо-Кавказский федеральный',
    r'Дальневосточный федеральный'
])
districts_mask = df['region'].str.contains(rm_districts, case=False)
df= df[~districts_mask].reset_index(drop = True)

In [12]:
years = [col for col in df.columns if col.isdigit()]
df[years] = df[years].astype("Int64")

In [13]:
def get_min_age(age):
    numbers = re.findall(r'\d+', age)
    return int(numbers[0]) if numbers else None
    
def get_max_age(age):
    numbers = re.findall(r"\d+", age)
    return int(numbers[-1]) if numbers else None
    
def categorize_age(age_str):
    age_str = age_str.lower()
    numbers = re.findall(r'\d+', age_str)
    words = re.findall(r'\b[а-яА-ЯёЁ]+\b', age_str, flags=re.IGNORECASE)
    if "-" in age_str and len(words) == 1:
        age_diff = int(numbers[1]) - int(numbers[0])
        return 2 if age_diff == 4 else 3
    elif "всего" in age_str or not numbers or len(words) > 3:
        return 4
    else:
        return 1

df['min_age']  = df['age'].apply(get_min_age)
df['max_age'] = df['age'].apply(get_max_age)
df['age_category'] = df['age'].apply(categorize_age)

In [14]:
#Add mid-year value
min_year = min([int(col) for col in df.columns if col.isdigit()])
df_mid = pd.DataFrame()

for col in df.columns:
    if col.isdigit():
        if int(col) > min_year:
            df_mid[f"{col}mid"] = (df[f"{col}"] + df[f"{int(col)-1}"])/2
        df_mid[f"{col}end"] = df[col]
    else:
        df_mid[col] = df[col]
df_mid.head()

,region,age,settlement,1990end,1991mid,1991end,1992mid,1992end,1993mid,1993end,...,2022end,2023mid,2023end,2024mid,2024end,2025mid,2025end,min_age,max_age,age_category
0,Российская Федерация,Всего,все население,78549601,78683648.0,78817695,78866241.5,78914788,78907045.5,78899303,...,78601633,78478172.5,78354712,78294074.0,78233436,78194072.5,78154709,NaN,NaN,4
1,Российская Федерация,Всего,городское население,57983496,58143680.0,58303864,58285854.0,58267844,58073152.5,57878461,...,59487917,59420291.0,59352665,59332816.0,59312967,59356113.0,59399259,NaN,NaN,4
2,Российская Федерация,Всего,сельское население,20566105,20539968.0,20513831,20580387.5,20646944,20833893.0,21020842,...,19113716,19057881.5,19002047,18961258.0,18920469,18837959.5,18755450,NaN,NaN,4
3,Российская Федерация,0,все население,1035959,996503.5,957048,912860.0,868672,819721.5,770771,...,673494,653034.5,632575,621914.5,611254,602290.5,593327,0.0,0.0,1
4,Российская Федерация,0,городское население,731046,698696.5,666347,630439.5,594532,555098.0,515664,...,506226,490632.5,475039,466675.0,458311,453286.0,448261,0.0,0.0,1


In [15]:
#Fix federal districts

dfo_regions = [
    'Республика Бурятия',
    'Забайкальский край',
    'Республика Саха (Якутия)',
    'Камчатский край',
    'Приморский край',
    'Хабаровский край',
    'Амурская область',
    'Магаданская область',
    'Сахалинская область',
    'Еврейская автономная область',
    'Чукотский автономный округ'
]
dfo_mask = df_mid['region'].isin(dfo_regions)
dfo = df_mid[dfo_mask]
dfo_agg = dfo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
dfo_agg.insert(0, 'region', 'Дальневосточный федеральный округ')

In [16]:
sfo_regions = ['Республика Алтай',
    'Республика Тыва',
    'Республика Хакасия',
    'Алтайский край',
    'Красноярский край',
    'Иркутская область',
    'Кемеровская область - Кузбасс',
    'Новосибирская область',
    'Омская область',
    'Томская область']
sfo_mask = df_mid['region'].isin(sfo_regions)
sfo = df_mid[sfo_mask]
sfo_agg = sfo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
sfo_agg.insert(0, 'region', 'Сибирский федеральный округ')

In [17]:
ufo_pattern = '|'.join([
    r'Адыгея', 
    r'Калмыкия',
    r'Краснодарский',
    r'Астраханская',
    r'Волгоградская',
    r'Ростовская',
    r'Крым',
    r'Севастополь'
])

skfo_pattern = '|'.join([
    r'Дагестан',
    r'Ингушетия',
    r'Кабардин',
    r'Балкар',
    r'Карачаев',
    r'Черкес',
    r'Осетия',
    r'Алания',
    r'Чечен',
    r'Ставрополь'
])

ufo_mask = df_mid['region'].str.contains(ufo_pattern, case = False, na = False)
ufo = df_mid[ufo_mask]
ufo_agg = ufo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
ufo_agg.insert(0, 'region', 'Южный федеральный округ')

skfo_mask = df_mid['region'].str.contains(skfo_pattern, case = False, na = False)
skfo = df_mid[skfo_mask]
skfo_agg = skfo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
skfo_agg.insert(0, 'region', 'Северо-Кавказский федеральный округ')

df_mid = pd.concat([df_mid, ufo_agg, skfo_agg, sfo_agg], ignore_index = True)


In [19]:
df_mid = df_mid.groupby('region', sort = False).apply(
    lambda x: x.sort_values(['age_category', 'min_age', 'max_age'])
).reset_index(drop = True)
df_mid.head()

C:\Users\User\AppData\Local\Temp\ipykernel_5108\2115352455.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_mid = df_mid.groupby('region', sort = False).apply(


,region,age,settlement,1990end,1991mid,1991end,1992mid,1992end,1993mid,1993end,...,2022end,2023mid,2023end,2024mid,2024end,2025mid,2025end,min_age,max_age,age_category
0,Российская Федерация,0,все население,1035959,996503.5,957048,912860.0,868672,819721.5,770771,...,673494,653034.5,632575,621914.5,611254,602290.5,593327,0.0,0.0,1
1,Российская Федерация,0,городское население,731046,698696.5,666347,630439.5,594532,555098.0,515664,...,506226,490632.5,475039,466675.0,458311,453286.0,448261,0.0,0.0,1
2,Российская Федерация,0,сельское население,304913,297807.0,290701,282420.5,274140,264623.5,255107,...,167268,162402.0,157536,155239.5,152943,149004.5,145066,0.0,0.0,1
3,Российская Федерация,1 год,все население,1132245,1082223.5,1032202,996674.5,961147,915514.0,869881,...,689192,681807.5,674423,653844.5,633266,623035.5,612805,1.0,1.0,1
4,Российская Федерация,1 год,городское население,799394,763825.0,728256,697132.0,666008,627874.0,589740,...,519345,513412.5,507480,491841.0,476202,468292.5,460383,1.0,1.0,1


In [ ]:
#Highlight "total" rows with red color
def highlight_total(row, color = 'red'):
    return [f"background-color: {color}" for _ in row]
df_styled = df_mid.style.apply(
    lambda row : highlight_total(row) if row['age'] == 'Всего' else ['']*len(row),
    axis = 1
)
df_styled

In [ ]:
#Save in a csv file
df_mid.drop(columns = ['age_category', 'min_age', 'max_age'], inplace = True)
df_mid.to_excel("sorted_data.xlsx", index = False)
df_styled.to_excel("sorted_data_highlighted.xlsx", index = False)

# Мужское население

In [22]:
men_population = FedStatIndicator(indicator_id = 31548)
df_men = men_population.get_processed_data()

In [23]:
df_men.head()

,region,age,settlement,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40
0,NaN,NaN,NaN,1990,1991,1992,1993,1994,1995,1996,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
1,NaN,NaN,NaN,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,...,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января
2,Российская Федерация,Всего,все население,69115480,69456051,69599904,69662391,69585409,69659152,69517865,...,68223758,68442150,68589452,68638602,68739894,NaN,68378428,68092712,67917353,67965219
3,Российская Федерация,Всего,городское население,50752687,51101222,51089862,50789946,50616235,50600552,50518815,...,50035322,50263930,50436035,50540327,50683205,NaN,50494479,50302898,50213973,50396541
4,Российская Федерация,Всего,сельское население,18362793,18354829,18510042,18872445,18969174,19058600,18999050,...,18188436,18178220,18153417,18098275,18056689,NaN,17883949,17789814,17703380,17568678


In [25]:
df_mem_raw = men_population.load_raw_indicator(data_type="excel")

ValueError: Ответ не содержит excel файла